In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/toy-encoding-data/token_type_ids.txt
/kaggle/input/toy-encoding-data/input_ids.txt
/kaggle/input/toy-encoding-data/attention.txt


In [2]:
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import torch
import sys
import csv
path_to_dataset = "/kaggle/input/toy-encoding-data/"

In [5]:
bs = 50
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
filename = 'pooled_'
    
    
print("begin loading tokens")
input_ids = np.loadtxt(path_to_dataset+"input_ids.txt")
token_type_ids = np.loadtxt(path_to_dataset+"token_type_ids.txt")
attention_mask = np.loadtxt(path_to_dataset+"attention.txt")
assert input_ids.shape == token_type_ids.shape
print("tokens loading done")
    
tensor_input_ids = torch.tensor(input_ids).to(torch.int64)
tensor_token_ids = torch.tensor(token_type_ids).to(torch.int64)
tensor_attention = torch.tensor(attention_mask).to(torch.int64)
    
dataset = TensorDataset(tensor_input_ids, tensor_token_ids, tensor_attention)
dataloader = DataLoader(dataset, batch_size=bs)
    
model = AutoModel.from_pretrained("bert-base-cased", output_hidden_states=True)
model.cuda()
model.eval()
    
count = 0
res = []
split = 1
for batch in dataloader:
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_token_type, b_input_mask = batch
        
    with torch.no_grad():
            
        outputs = model(b_input_ids, token_type_ids=b_token_type,
                        attention_mask=b_input_mask)
        hidden_states = outputs[2]
        sentence_embedding = torch.mean(hidden_states[-1], dim=1).squeeze()
        res.append(sentence_embedding.cpu().numpy())
        count += 1
        if count == 2000:
            np.save(filename+str(split)+".npy", np.stack(res, axis=0))
            res = []
            count = 0
            split += 1
    
    np.save(filename+str(split)+".npy", np.stack(res, axis=0))    

begin loading tokens
tokens loading done


Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]